# Tests4Py

In [1]:
from tests4py.api.logging import deactivate, debug
deactivate()
#debug()

tests4py :: INFO     :: Loading projects


In [2]:
from debugging_benchmark.tests4py_benchmark.repository import (
    #CalculatorBenchmarkRepository,
    #MarkUpBenchmarkRepository, 
    #ExpressionBenchmarkRepository,
    #MiddleBenchmarkRepository,
    #TheFuckBenchmarkRepository,
    PysnooperBenchmarkRepository,
    CookieCutterBenchmarkRepository,
)

In [3]:
repos = [
    #CalculatorBenchmarkRepository(),
    #ExpressionBenchmarkRepository(),
    #MiddleBenchmarkRepository(),
    #MarkUpBenchmarkRepository(),
    PysnooperBenchmarkRepository(),
    CookieCutterBenchmarkRepository(),
]

In [4]:
repos = [
    PysnooperBenchmarkRepository(),
    CookieCutterBenchmarkRepository(),
]

In [5]:
from typing import List
from debugging_framework.benchmark.program import BenchmarkProgram

programs: List[BenchmarkProgram] = []
List[BenchmarkProgram]
for repo in repos:
    tmp: List[BenchmarkProgram] = repo.build()
    for prog in tmp:
        programs.append(prog)

In [6]:
# Display all benchmark programs
programs

[Tests4PyBenchmarkProgram(pysnooper_2),
 Tests4PyBenchmarkProgram(pysnooper_3),
 Tests4PyBenchmarkProgram(cookiecutter_2),
 Tests4PyBenchmarkProgram(cookiecutter_3),
 Tests4PyBenchmarkProgram(cookiecutter_4)]

## Verify Build

In [7]:
# Check for first Program

In [8]:
program = programs[0]

In [9]:
program.get_grammar()

{'<start>': ['<options>'],
 '<options>': [' ', '<flag><op>'],
 '<flag>': ['<overwrite><thread_info>'],
 '<op>': ['<output><depth><prefix><watch><custom_repr>'],
 '<sep>': [' ', '\n'],
 '<output>': ['-o<sep>', '-o<path><sep>', '-o<sep><path><sep>', ''],
 '<depth>': ['-d<number><sep>',
  '-d<sep><number><sep>',
  '-d=<number><sep>',
  ''],
 '<prefix>': ['-p<str_ascii><sep>',
  '-p<sep><str_ascii><sep>',
  '-p=<str_ascii><sep>',
  ''],
 '<watch>': ['-w<variable_list><sep>',
  '-w<sep><variable_list><sep>',
  '-w=<variable_list><sep>',
  ''],
 '<custom_repr>': ['-c<predicate_list><sep>',
  '-c<sep><predicate_list><sep>',
  '-c=<predicate_list><sep>',
  ''],
 '<overwrite>': ['-O<sep>', ''],
 '<thread_info>': ['-T<sep>', ''],
 '<path>': ['<location>', '<location>.<str_ascii>'],
 '<location>': ['<str_ascii>', '<path>/<str_ascii>'],
 '<variable_list>': ['<variable>', '<variable_list>,<variable>'],
 '<variable>': ['<name>', '<variable>.<name>'],
 '<name>': ['<letter><chars>'],
 '<chars>': ['', 

In [10]:
oracle = program.get_oracle()

In [11]:
for inp in program.get_initial_inputs():
    print(inp, oracle(inp))

-otest.log  (<OracleResult.PASSING: 'PASSING'>, None)
  (<OracleResult.PASSING: 'PASSING'>, None)
-T -d1  (<OracleResult.PASSING: 'PASSING'>, None)
-o -d1  (<OracleResult.PASSING: 'PASSING'>, None)
-O -otest.log  (<OracleResult.PASSING: 'PASSING'>, None)
-d1 -wx  (<OracleResult.PASSING: 'PASSING'>, None)
-wy  (<OracleResult.PASSING: 'PASSING'>, None)
-otest.log -wx  (<OracleResult.PASSING: 'PASSING'>, None)
-ptest  (<OracleResult.PASSING: 'PASSING'>, None)
-ptest -wx  (<OracleResult.PASSING: 'PASSING'>, None)
-otest.log -cint=str  (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
-cint=str  (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
-T -d1 -cint=repr  (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
-o -d1 -cbool=str  (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
-O -otest.log -cint=repr,bool=str  (<Oracl

In [12]:
# Verify grammar

In [13]:
from debugging_framework.fuzzingbook.grammar import is_valid_grammar

for program in programs:
    grammar = program.get_grammar()
    assert is_valid_grammar(grammar), f"Grammar not valid for Project {program}"

In [14]:
# Verify initial inputs

In [15]:
from isla.parser import EarleyParser, tree_to_string

for program in programs:
    grammar = program.get_grammar()
    parser = EarleyParser(grammar)
    for inp in program.get_initial_inputs():
        for tree in parser.parse(inp):
            assert tree_to_string(tree) == str(inp), f"Could not parse {inp} for Project {program}"

In [16]:
# Verify Oracles

In [17]:
from debugging_framework.input.oracle import OracleResult

for program in programs:
    oracle = program.get_oracle()
    print(program)
    for inp in program.get_initial_inputs():
        test_result, exception = oracle(inp)
        
        # only display the first 50 chars as inputs are quite long for cookiecutter; shortend inputs [:50]
        print((inp.encode()[:50]).ljust(60), test_result)

Tests4PyBenchmarkProgram(pysnooper_2)
b'-otest.log                                                  ' PASSING
b'                                                            ' PASSING
b'-T -d1                                                      ' PASSING
b'-o -d1                                                      ' PASSING
b'-O -otest.log                                               ' PASSING
b'-d1 -wx                                                     ' PASSING
b'-wy                                                         ' PASSING
b'-otest.log -wx                                              ' PASSING
b'-ptest                                                      ' PASSING
b'-ptest -wx                                                  ' PASSING
b'-otest.log -cint=str                                        ' FAILING
b'-cint=str                                                   ' FAILING
b'-T -d1 -cint=repr                                           ' FAILING
b'-o -d1 -cbool=str       

In [18]:
from debugging_framework.input.oracle import OracleResult

for program in programs:
    oracle = program.get_oracle()
    for inp in program.get_failing_inputs():
        test_result, exception = oracle(inp)
        assert test_result == OracleResult.FAILING, f"Unexpected behavior for Input '{inp}' for Project {program}"

In [19]:
from debugging_framework.input.oracle import OracleResult

for program in programs:
    oracle = program.get_oracle()
    for inp in program.get_passing_inputs():
        test_result, exception = oracle(inp)
        assert test_result == OracleResult.PASSING, f"Unexpected behavior for Input '{inp}' for Project {program}"

In [20]:
from debugging_framework.fuzzingbook.fuzzer import GrammarFuzzer, is_valid_grammar

# First Program (Pysnooper 2)
program = programs[0]


grammar = program.get_grammar()
oracle = program.get_oracle()

fuzzer = GrammarFuzzer(grammar, max_nonterminals=20)

for _ in range(20):
    inp = fuzzer.fuzz()
    print(inp.encode(), oracle(inp))

b' ' (<OracleResult.PASSING: 'PASSING'>, None)
b'-O\n-T\n-d7 -w\nt__.p_1_,K.H9o_.Q.q3__.D.U.nr.T_,s3.L11_.R_\n-c=int=int ' (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
b'-T\n-o\n-d9\n-p J -w M_.AQ435_.jy\n-c bool=repr,int=str\n' (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
b' ' (<OracleResult.PASSING: 'PASSING'>, None)
b'-T\n-d=2\n-p\nfq -cbool=int,bool=int,bool=int\n' (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
b' ' (<OracleResult.PASSING: 'PASSING'>, None)
b' ' (<OracleResult.PASSING: 'PASSING'>, None)
b'-o -d=83 -p=m\n-c float=repr ' (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
b'-T -p\nW -c\nbool=int,float=int,str=int,float=int ' (<OracleResult.FAILING: 'FAILING'>, Tests4PySubjectException('An Exception was triggered.'))
b' ' (<OracleResult.PASSING: 'PASSING'>, None)
b'-O\n-T\n-o\n9\n-d99\n-p